# SQL con clase

*The single biggest challenge to learning SQL programming is unlearning procedureal programming.*
-- Joe Celko

En este notebook, exploraremos diversos estilos y prácticas para escribir consultas SQL enfocadas en el análisis de datos. Las consultas analíticas se distinguen de las consultas convencionales en que están diseñadas específicamente para trabajar sobre grandes volúmenes de datos y tienen a ser más complejas.

## Setup

Primero vamos a configurar la notebook y definir las variables principales.

Instalamos los paquetes necesarios. Este paso se puede omitir si ya lo realizó en su entorno, sin embargo si usa **Colab** va a ser necesario ejecutarlo.

In [1]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.5 MB/s eta 0:00:00


Configuracion la extensión de %sql para el notebook, lo que nos va a permitir usar lenguaje SQL directamente sobre una celda

In [6]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Y finalmente cargamos el dataset en la base de datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_path = '/home/aleb/DMEyF/2024/datos/' # '/content/drive/MyDrive/DMEyF/2024/datos/'
dataset_path = '/content/drive/Othercomputers/Mi Mac/Documents/'
dataset_file = 'competencia_01_crudo.csv'

In [7]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## WITH



Un escenario común es necesitar realizar una consulta a no a una tabla, sino a otra consulta. Una forma tradicional de hacerlo es usando es:
* Creando una tabla temporal con esa consulta
* Usando `selects from select`.

Esta necesidad es muy común, pero como todo en la vida hay peros.
- Crear una tabla en una base de datos que va a ser usada UNA SOLA VEZ ensucia los esquemas con un montón de tablas que no son útiles y además del desorden ocupan espacio innecesario
- El `select from select`, debe ser de las cosas peor vistas por los administradores de BBDD. Tienen sus razones técnicas, en general la recomendación es no usarlo a menos que uno conozca bien como funciona la base de datos por dentro, ya que hay casos donde terminan ocasionando muchos problemas

Y que hacer? bueno hay una clausula en **SQL** llamada `WITH`, que nos permite crear esas (plural) tablas para ser usadas en la **query** principal. Ejemplo si queremos comparar el valor de una variable con su promedio a través de todos los meses.


In [8]:
%%sql
with promedios as (
    select
        numero_de_cliente
        , avg(mrentabilidad) as avg_mrentabilidad
    from competencia_01_crudo
    group by numero_de_cliente
) select
    cp.numero_de_cliente
    , cp.foto_mes
    , if(cp.mrentabilidad > p.avg_mrentabilidad, 'mayor', 'menor') as comparacion
from competencia_01_crudo cp
join promedios p using (numero_de_cliente)


,numero_de_cliente,foto_mes,comparacion
0,249221323,202101,menor
1,249227600,202101,menor
2,249234235,202101,menor
3,249244449,202101,mayor
4,249244739,202101,menor
...,...,...,...
978434,1598368433,202106,menor
978435,1598419415,202106,menor
978436,1598425905,202106,menor
978437,1598470389,202106,menor


- Usar esa clausula tiene muchos beneficios, entre ellos sumar legibilidad de código y le permite al motor de **SQL** optimizar mejor, ya que entiende todo lo que se buscar hacer y planifica como resolverlo en el menor tiempo posible.

**NOTA**: Hay un tipo de tabla que se llaman temporales, que se borran solas una vez que uno se desconecta de la base de datos. Es una mejor alternativa frente al `CREATE`, sin embargo si la tabla que se esta creando se usa una sola vez es conveniente usar el `WITH` porque permite optimizar la ejecución. Si esa tabla va a ser usada mas de una vez dentro del proceso, las tablas temporales son la mejor opción.

# Funciones Analíticas

Las funciones analíticas en SQL son un conjunto de funciones que te permiten realizar cálculos avanzados sobre un conjunto de filas relacionadas dentro de una consulta conservando las filas individuales y calculando sobre ventanas de datos relacionadas.

Veamos un ejemplo para que quede más claro:

In [10]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
from competencia_01_crudo
where numero_de_cliente = 918147740
order by foto_mes

,numero_de_cliente,foto_mes,mrentabilidad
0,918147740,202101,276.74
1,918147740,202102,-735.18
2,918147740,202103,-417.20
3,918147740,202104,-694.08
4,918147740,202105,-1692.76
5,918147740,202106,-897.20


In [11]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
    , lead(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_1
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_1
    , lead(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_2
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_2
from competencia_01_crudo
where numero_de_cliente = 918147740
order by foto_mes



,numero_de_cliente,foto_mes,mrentabilidad,mrentabilidad_mas_1,mrentabilidad_menos_1,mrentabilidad_mas_2,mrentabilidad_menos_2
0,918147740,202101,276.74,-735.18,NaN,-417.20,NaN
1,918147740,202102,-735.18,-417.20,276.74,-694.08,NaN
2,918147740,202103,-417.20,-694.08,-735.18,-1692.76,276.74
3,918147740,202104,-694.08,-1692.76,-417.20,-897.20,-735.18
4,918147740,202105,-1692.76,-897.20,-694.08,NaN,-417.20
5,918147740,202106,-897.20,NaN,-1692.76,NaN,-694.08


* Qué paso?
* ¿Cómo esta construyendo las nuevas variables?
* ¿ Nos puede ayudar a construir targets ?

¿Dondé puedo leer un poco más de esta magia negra? https://duckdb.org/docs/sql/window_functions.html

## Los casos raros

Que pasa con el cliente **930544172** y la foto **202103**


In [12]:
%%sql
select
    numero_de_cliente
    , foto_mes
from competencia_01_crudo
where numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes
0,930544172,202101
1,930544172,202102


* ¿Cuál debería ser la clase para cada periodo?

### Una pista

Podemos generar todas las posibles combinaciones de clientes y periodos de manera muy simple

In [13]:
%%sql
with periodos as (
  select distinct foto_mes from competencia_01_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_01_crudo
)
select numero_de_cliente, foto_mes from clientes cross join periodos
where numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes
0,930544172,202101
1,930544172,202102
2,930544172,202103
3,930544172,202104
4,930544172,202105
5,930544172,202106


Solo nos queda saber si estuvo o no el banco el cliente en ese periodo y armar el

In [ ]:
%%sql
with periodos as (
    select distinct foto_mes from competencia_01_crudo -- Esto también se puede hacer con secuencias
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
)
select
    t.numero_de_cliente
    , t.foto_mes
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
from todo t
left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
where t.numero_de_cliente = 930544172
order by foto_mes

Antes de continuar, tomémonos un momento para reflexionar. Con todas estas piezas, ¿cómo podemos ensamblar el target?

Una vez que lo tengamos claro sobre el papel, proceda a completar el código que sigue.

In [ ]:
%%sql
create or replace table competencia_01 as
with periodos as (
    select distinct foto_mes from competencia_01_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , null as clase_ternaria -- Replazar null por la lógica que genera el target
    from todo t
    left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

Revisamos que todo salga como esperamos

In [ ]:
%sql select count(*) from competencia_01

Y vemos la cardinalidad de las clases por periodo

In [ ]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

Y finalmente almacenamos el nuevo dataset para ser usado a continuación

In [ ]:
%%sql COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)


## El pato del amor

DuckDB no es una base de datos SQL estándar; su sintaxis moderna facilita significativamente la escritura de consultas. A continuación, te menciono algunas características destacadas:

+ Uso de variables: En DuckDB, no es necesario reescribir las variables; puedes utilizar tus variables una vez definidas, lo que simplifica el código.

+ Función IF: DuckDB soporta la función IF(condición, valor_si_verdadero, valor_si_falso), lo que hace que tu código sea más claro y legible en comparación con el uso de CASE WHEN.

+ Exclusión de campos: Puedes utilizar SELECT * EXCLUDE(field1, ...) para excluir campos específicos de la lista de selección, lo que ahorra tiempo y esfuerzo.

+ Alias en cláusulas: Es posible utilizar alias en las cláusulas WHERE, GROUP BY y HAVING, lo que añade flexibilidad a la escritura de consultas.

+ Uso de alias en cálculos: DuckDB permite el uso de alias dentro del cálculo de otros campos, facilitando la manipulación y transformación de datos en una consulta.

Estas son solo algunas de las capacidades que hacen de DuckDB una herramienta poderosa. Te recomiendo leer la documentación y el blog oficial para explorar más a fondo su potencial.